In [2]:
import pandas as pd
from datetime import date
import numpy as np


df3 = pd.DataFrame(index=range(90), columns=
                   ['year', 'state', 'apple_sold', 'apple_stolen', 'pen_sold', 'pen_stolen'])
year = 2006
for a in range(90):
    df3.iloc[a].year = year
    df3.iloc[a].apple_sold = 0
    df3.iloc[a].apple_stolen = 0
    df3.iloc[a].pen_sold = 0
    df3.iloc[a].pen_stolen = 0
    year = year + 1
    if year == 2016:
        year = 2006
id_sell = 0
id_st = 0


In [3]:
def work(statename):

    global id_sell
    global id_st
    filename_sup = 'out/input/MS-b1-supply.csv'
    filename_sup = filename_sup[:13] + statename + filename_sup[15:]
    filename_inv = 'out/input/MS-b1-inventory.csv'
    filename_inv = filename_inv[:13] + statename + filename_inv[15:]
    filename_sell = 'out/input/MS-b1-sell.csv'
    filename_sell = filename_sell[:13] + statename + filename_sell[15:]

    supply_df = pd.read_csv(filename_sup, sep=',', parse_dates=['date'],
                            index_col='date')

    inventory_df = pd.read_csv(filename_inv, sep=',', parse_dates=['date'],
                               index_col='date')

    sell_df = pd.read_csv(filename_sell, sep=',', parse_dates=['date'],
                          index_col='date')

    df3.iloc[id_sell].state = statename
    d0 = supply_df.iloc[0].name
    d1 = inventory_df.iloc[inventory_df.shape[0] - 1].name
    df = pd.DataFrame(index=pd.date_range(
        d0, d1, freq='D'), columns=['apple', 'pen'])
    df.index.name = 'date'
    steal_df = inventory_df.copy()

    apple_sold = 0
    pen_sold = 0
    apple_stolen = 0
    pen_stolen = 0

    i1 = 0  # наш файл
    i2 = 0  # supply
    i3 = 0  # sell
    i4 = 0  # inventory
    df.iloc[0].apple = 0
    df.iloc[0].pen = 0
    while i1 != df.shape[0]:
        if i2 != supply_df.shape[0]:
            if df.iloc[i1].name == supply_df.iloc[i2].name:
                df.iloc[i1].apple = df.iloc[i1].apple + supply_df.iloc[i2].apple
                df.iloc[i1].pen = df.iloc[i1].pen + supply_df.iloc[i2].pen
                i2 = i2 + 1

        if i3 != sell_df.shape[0]:
            while sell_df.iloc[i3].name == df.iloc[i1].name:
                if sell_df.iloc[i3].sku_num.find('ap') != -1:
                    df.iloc[i1].apple = df.iloc[i1].apple - 1
                    df3.iloc[id_sell].apple_sold = df3.iloc[id_sell].apple_sold + 1
                if sell_df.iloc[i3].sku_num.find('pe') != -1:
                    df.iloc[i1].pen = df.iloc[i1].pen - 1
                    df3.iloc[id_sell].pen_sold = df3.iloc[id_sell].pen_sold + 1
                i3 = i3 + 1
                if i3 == sell_df.shape[0]:
                    id_sell = id_sell + 1
                    break
                if sell_df.iloc[i3].name.year != df3.iloc[id_sell].year:
                    id_sell = id_sell + 1
                    df3.iloc[id_sell].state = statename

        if i4 != inventory_df.shape[0]:
            if df3.iloc[id_st].year != steal_df.iloc[i4].name.year:
                id_st = id_st + 1
            if df.iloc[i1].name == inventory_df.iloc[i4].name:
                steal_df.iloc[i4].apple = df.iloc[i1].apple - inventory_df.iloc[i4].apple
                df3.iloc[id_st].apple_stolen = df3.iloc[id_st].apple_stolen + steal_df.iloc[i4].apple
                steal_df.iloc[i4].pen = df.iloc[i1].pen - inventory_df.iloc[i4].pen
                df3.iloc[id_st].pen_stolen = df3.iloc[id_st].pen_stolen + steal_df.iloc[i4].pen
                df.iloc[i1].apple = inventory_df.iloc[i4].apple
                df.iloc[i1].pen = inventory_df.iloc[i4].pen
                i4 = i4 + 1

        i1 = i1 + 1
        if i1 != df.shape[0]:
            df.iloc[i1].apple = df.iloc[i1 - 1].apple
            df.iloc[i1].pen = df.iloc[i1 - 1].pen

    filenameout_d = 'MS-b1-daily.csv'
    filenameout_d = filenameout_d[:3] + statename + filenameout_d[5:]
    filenameout_s = 'MS-b1-steal.csv'
    filenameout_s = filenameout_s[:3] + statename + filenameout_s[5:]
    df.to_csv(filenameout_d)
    steal_df.to_csv(filenameout_s)

    return 1


In [4]:
work('b1')
work('b2')
work('m1')
work('m2')
work('s1')
work('s2')
work('s3')
work('s4')

del df3['Unnamed: 0']
df3.index=df3['year']
del df3['year']
df3.to_csv('agregate.csv')

KeyboardInterrupt: 